In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

train=pd.read_csv("engtamilTrain.csv")
train=train.drop(["Unnamed: 0"],axis=1)
english_sentences=train["en"]
tamil_sentence=train['ta']
english_sentences=english_sentences.head(1000)
tamil_sentences=tamil_sentence.head(1000)

In [3]:
#Function to add SOS and EOS to the statement
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

train=pd.read_csv("engtamilTrain.csv")
train=train.drop(["Unnamed: 0"],axis=1)
english_sentences=train["en"]
tamil_sentence=train['ta']
english_sentences=english_sentences.head(1000)
tamil_sentences=tamil_sentence.head(1000)

def addSosEos(seriesSentence):
    # Define the <SOS> and <EOS> tokens
    sos_token = "<SOS>"
    eos_token = "<EOS>"

    # Add <SOS> and <EOS> tokens to each statement
    statements_with_tokens = [f"{sos_token} {statement} {eos_token}" for statement in seriesSentence]

    english_sent=[]
    # Print the statements with tokens
    for statement in statements_with_tokens:
        english_sent.append(statement)
      #  print(statement)
    return english_sent


In [4]:
english_sent_SE=addSosEos(english_sentences)
tamil_sent_SE=addSosEos(tamil_sentences)

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Tokenize the English and Tamil sentences
english_tokenizer = Tokenizer(filters="")
english_tokenizer.fit_on_texts(english_sent_SE)
english_vocab_size = len(english_tokenizer.word_index) + 1
english_sequences = english_tokenizer.texts_to_sequences(english_sent_SE)

In [7]:
tamil_tokenizer = Tokenizer(filters="")
tamil_tokenizer.fit_on_texts(tamil_sent_SE)
tamil_vocab_size = len(tamil_tokenizer.word_index) + 1
tamil_sequences = tamil_tokenizer.texts_to_sequences(tamil_sent_SE)

In [8]:
max_input_seq_length=20
max_output_seq_length=20

In [9]:
# Pad sequences to a fixed length
input_sequences = pad_sequences(english_sequences, maxlen=max_input_seq_length, padding='post')
output_sequences = pad_sequences(tamil_sequences, maxlen=max_output_seq_length, padding='post')

In [10]:
# Prepare the decoder input and output sequences for teacher forcing
decoder_input_sequences = np.zeros_like(output_sequences)
decoder_input_sequences[:, 1:] = output_sequences[:, :-1]
decoder_input_sequences[:, 0] = tamil_tokenizer.word_index['<sos>']

decoder_output_sequences = np.eye(tamil_vocab_size)[output_sequences]


In [11]:
from gensim.models import Word2Vec

eng_model = Word2Vec.load('engmodel.bin')
tam_model = Word2Vec.load('tammodel.bin')


C:\Users\Arun\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [12]:
def create_embedding_matrix(word2vec_model, tokenizer, vocab_size):
    embedding_matrix = np.zeros((vocab_size, word2vec_model.vector_size))
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = word2vec_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            pass  # Words not found in the embedding index will be all zeros
    return embedding_matrix

eng_embedding_matrix = create_embedding_matrix(eng_model, english_tokenizer, english_vocab_size)
tam_embedding_matrix = create_embedding_matrix(tam_model, tamil_tokenizer, tamil_vocab_size)


In [13]:
eng_embedding_matrix.shape

(6979, 100)

In [14]:
tam_embedding_matrix.shape

(9922, 100)

In [15]:
def create_seq2seq_model(input_vocab_size, output_vocab_size, input_seq_length, output_seq_length, hidden_units, eng_embedding_matrix, tam_embedding_matrix):
    # Encoder
    encoder_inputs = Input(shape=(input_seq_length,))
    encoder_embedding = Embedding(input_vocab_size, hidden_units, weights=[eng_embedding_matrix], trainable=False)(encoder_inputs)
    encoder_lstm, encoder_state_h, encoder_state_c = LSTM(hidden_units, return_state=True)(encoder_embedding)

    # Decoder
    decoder_inputs = Input(shape=(output_seq_length,))
    decoder_embedding = Embedding(output_vocab_size, hidden_units, weights=[tam_embedding_matrix], trainable=False)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_state_h, encoder_state_c])
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model


In [16]:
# Convert target_sequences to one-hot encoded format
target_sequences = tf.keras.utils.to_categorical(output_sequences, num_classes=tamil_vocab_size)


In [17]:
model = create_seq2seq_model(english_vocab_size, tamil_vocab_size, max_input_seq_length, max_output_seq_length, 100, eng_embedding_matrix, tam_embedding_matrix)

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
# Fit the model to the data
batch_size = 32
epochs = 500
model.fit([input_sequences, output_sequences], decoder_output_sequences, batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 154ms/step - accuracy: 0.2149 - loss: 9.1053 - val_accuracy: 0.2722 - val_loss: 8.2737
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - accuracy: 0.2821 - loss: 7.5931 - val_accuracy: 0.2660 - val_loss: 7.4754
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.2713 - loss: 6.5534 - val_accuracy: 0.2722 - val_loss: 7.5368
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - accuracy: 0.2745 - loss: 6.3863 - val_accuracy: 0.2722 - val_loss: 7.5962
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 132ms/step - accuracy: 0.2670 - loss: 6.3792 - val_accuracy: 0.2722 - val_loss: 7.5909
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.2745 - loss: 6.2389 - val_accuracy: 0.2722 - val_loss: 7.5679
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.2675 - loss: 6.2398 - val_accuracy: 0.2720 - val_loss: 7.5617
Epoch 8/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.2852 - loss: 6.0689 - val_accu

25/25 [==============================] - 5s 189ms/step - loss: 2.0486 - accuracy: 0.5856 - val_loss: 11.3001 - val_accuracy: 0.2765
Epoch 340/500
25/25 [==============================] - 4s 170ms/step - loss: 1.9937 - accuracy: 0.6076 - val_loss: 11.3216 - val_accuracy: 0.2820
Epoch 341/500
25/25 [==============================] - 4s 166ms/step - loss: 1.9869 - accuracy: 0.6046 - val_loss: 11.3037 - val_accuracy: 0.2763
Epoch 342/500
25/25 [==============================] - 4s 173ms/step - loss: 1.9523 - accuracy: 0.6202 - val_loss: 11.3210 - val_accuracy: 0.2765
Epoch 343/500
25/25 [==============================] - 4s 169ms/step - loss: 1.9370 - accuracy: 0.6261 - val_loss: 11.3253 - val_accuracy: 0.2797
Epoch 344/500
25/25 [==============================] - 4s 180ms/step - loss: 1.9421 - accuracy: 0.6211 - val_loss: 11.3346 - val_accuracy: 0.2805
Epoch 345/500
25/25 [==============================] - 4s 165ms/step - loss: 1.9383 - accuracy: 0.6209 - val_loss: 11.3452 - val_accuracy:

25/25 [==============================] - 4s 150ms/step - loss: 1.3014 - accuracy: 0.7710 - val_loss: 11.9418 - val_accuracy: 0.2720
Epoch 452/500
25/25 [==============================] - 4s 152ms/step - loss: 1.2967 - accuracy: 0.7700 - val_loss: 11.9505 - val_accuracy: 0.2760
Epoch 453/500
25/25 [==============================] - 4s 153ms/step - loss: 1.2913 - accuracy: 0.7699 - val_loss: 11.9747 - val_accuracy: 0.2715
Epoch 454/500
25/25 [==============================] - 4s 151ms/step - loss: 1.2841 - accuracy: 0.7724 - val_loss: 11.9704 - val_accuracy: 0.2705
Epoch 455/500
25/25 [==============================] - 4s 152ms/step - loss: 1.2796 - accuracy: 0.7742 - val_loss: 11.9769 - val_accuracy: 0.2730
Epoch 456/500
25/25 [==============================] - 4s 154ms/step - loss: 1.2728 - accuracy: 0.7758 - val_loss: 11.9837 - val_accuracy: 0.2713
Epoch 457/500
25/25 [==============================] - 4s 153ms/step - loss: 1.2729 - accuracy: 0.7760 - val_loss: 11.9774 - val_accuracy:

In [20]:
# Preprocessing the input
input_sentence = "<sos>Finally, the columnist fails to tell us who among the political leaders of the bourgeoisie, past and present, he counts among the paragons of morality<eos>"

# Convert the input sentence to sequence
input_sequence = english_tokenizer.texts_to_sequences([input_sentence])

# Pad the statement to the maximum input sequence length
input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_length, padding='post')

# Generate predictions
predictions = model.predict([input_sequence, np.zeros((1, max_output_seq_length))])

# Convert predictions to tokens
predicted_tokens = np.argmax(predictions, axis=-1)[0]

# Create index to word mapping for Tamil vocabulary
tamil_index_word = {i: w for w, i in tamil_tokenizer.word_index.items()}


# Convert tokens to text
decoded_sentence = []
for token in predicted_tokens:
    if token == 0:  # Assuming 0 is the padding token
        continue
    word = tamil_index_word.get(token)
    if word == '<eos>':
        break
    if word is not None:
        decoded_sentence.append(word)
    else:
        decoded_sentence.append('<unk>')

# Join the words to form the decoded statement
decoded_statement = ' '.join(decoded_sentence)

# Print the decoded statement
print(decoded_statement)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step
ஆக்கிரமிப்பு வேகமாக ஈராக் பரவியதால் ஜேர்மனிய தோவேக்கு பிரதியுத்தரமாக: ஈசாயின் மகனை நோபிலிருக்கிகிற


In [ ]:
predictions

In [ ]:
predicted_tokens

In [ ]:
input_sequence

In [ ]:
input_sequence